In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wrangle_data as wd

%matplotlib inline

In [2]:
df = wd.run('2020-05-02', '2020-06-27')
df_c = wd.merge_complex(df)

In [3]:
key = df_c[['complex_id', 'station_x']].drop_duplicates()

In [4]:
#aggregate by complex id and date
df_daily = wd.agg_by(df_c, 'date', 'complex')

#create a turnstile count/complex column
df_cc = df_c.groupby('complex_id')['tuid'].nunique().to_dict()
df_daily['ts_count'] = df_daily.complex_id.map(df_cc)

#calculate density
df_daily['density'] = df_daily['traffic'] / df_daily['ts_count']

#calculate average traffic and density figures for entire date range
df_daily_avg = df_daily.groupby('complex_id')[['traffic', 'density']].mean()

In [5]:
#Calculate priority score
traf_top = df_daily_avg['traffic'].max()
dens_top = df_daily_avg['density'].max()
traffic_weight, density_weight = 0, 0
tra_score = (df_daily_avg['traffic'] / traf_top) 
dens_score = (df_daily_avg['density'] / dens_top) 
df_daily_avg['priority'] = (tra_score + traffic_weight) * (dens_score + density_weight)


In [6]:
#normalize priority score
p_min = df_daily_avg['priority'].min()
p_max = df_daily_avg['priority'].max()
p_range = p_max - p_min
df_daily_avg['priority']=(df_daily_avg['priority']-p_min)/(p_range)


In [7]:
#Export data to csv's so John can implement mapping
df_daily_avg['traffic'].to_csv(path_or_buf='complex_traffic.csv', index=True)
df_daily_avg['density'].to_csv(path_or_buf='complex_density.csv', index=True)
df_daily_avg['priority'].to_csv(path_or_buf='complex_priority.csv', index=True)

In [8]:
key.query('complex_id == 392')

,complex_id,station_x
1516627,392.0,125 ST


In [9]:
df_daily_avg.sort_values('priority', ascending=False).head(20)

,traffic,density,priority
complex_id,,,
392.0,12128.015873,866.286848,1.000000
611.0,30704.555556,310.147026,0.906396
616.0,15433.761905,571.620811,0.839705
450.0,8769.142857,974.349206,0.813239
602.0,18599.841270,453.654665,0.803121
447.0,14924.174603,552.747208,0.785169
278.0,12844.476190,611.641723,0.747754
345.0,11536.873016,640.937390,0.703799
451.0,9399.174603,783.264550,0.700717
